> ### EEE4423: Deep Learning Lab

# Final Project: Long-tail Visual Recognition for Image Classification

In [13]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import os.path as osp
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import optuna
import joblib


from misc.project.utils import resnet18, IMBALANCECIFAR10_V, IMBALANCECIFAR100_V, compute_accuracy

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [16]:
DATASET = 'CIFAR10' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'exp' #['exp', 'step']
IMB_FACTOR = 0.01 #[0.1, 0.01]
SAVE_DIR = 'logs/exp-0.1' 

BATCH_SIZE = 128

os.makedirs(osp.join(SAVE_DIR), exist_ok=True)

In [17]:
# num_cls = 10, 100 (CIFAR10, CIFAR100)
# val_num_each_cls = 1000, 100 (CIFAR10, CIFAR100)

def for_split_set(num_cls, val_num_each_cls, samples_of_each_cls):
    val_idx_list = []
    for i in range(num_cls):
        for j in range(val_num_each_cls):
            index = np.random.randint(i*0, (i+1)*samples_of_each_cls) # 0~4999의 숫자를 랜덤으로 하나 뽑는다.
            while index in val_idx_list: # index가 이미 리스트에 있다면 다시 뽑는다.
                index = np.random.randint(i*0, (i+1)*samples_of_each_cls) # 0~4999의 숫자를 랜덤으로 하나 뽑는다.
            val_idx_list.append(index)

    train_idx = np.arange(50000)
    train_idx_list = list(np.delete(train_idx, val_idx_list))

    for i in val_idx_list:
        if i in train_idx_list:
            print("중복이 있습니다!")
    
    return val_idx_list, train_idx_list 

In [18]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [19]:
def get_loaders():
    # inbalance 된거니까 쓰면됨
    if DATASET == 'CIFAR10':
        val_idx, train_idx = for_split_set(10, 1000, 5000)
        train_dataset = IMBALANCECIFAR10_V(root='../dataset/project', imb_type=IMB_TYPE, imb_factor=IMB_FACTOR, sub_idx=val_idx, train=True, download=True, transform=transform_train)
        val_dataset = IMBALANCECIFAR10_V(root='../dataset/project', imb_factor=1, train=True, download=True, sub_idx=train_idx, transform=transform_test)
    elif DATASET == 'CIFAR100':
        val_idx, train_idx = for_split_set(100, 100, 500)
        train_dataset = IMBALANCECIFAR100_V(root='../dataset/project', imb_type=IMB_TYPE, imb_factor=IMB_FACTOR, sub_idx=val_idx, train=True, download=True, transform=transform_train)
        val_dataset = IMBALANCECIFAR100_V(root='../dataset/project', imb_factor=1, train=True, download=True, sub_idx=train_idx, transform=transform_test)

    cls_num_list = train_dataset.get_cls_num_list()
    print('cls num list(train_dataset):')
    print(cls_num_list)

    cls_num_list = val_dataset.get_cls_num_list()
    print('cls num list(val_dataset):')
    print(cls_num_list)
    num_classes = len(cls_num_list)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=BATCH_SIZE, shuffle=True,
        num_workers=4, drop_last=True)

    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=100, shuffle=False,
        num_workers=4, )

    return train_loader, val_loader, num_classes

In [21]:
class ResNet18(nn.Module):
    def __init__(self, num_classes):
        super(ResNet18, self).__init__()
        self.backbone = resnet18()
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = F.adaptive_max_pool2d(x, 1)
        x = x.view(batch_size, -1)
        pred = self.classifier(x)
        return pred

In [22]:
def train(EPOCHS, model, train_loader, criterion, optimizer, scheduler):

    for epoch in range(EPOCHS):
        loss_history = []
        model.train()
        for batch_index, data in enumerate(train_loader):
            image, target = data
            image, target = image.cuda(), target.cuda()

            pred = model(image)
            loss = criterion(pred, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_history.append(loss.item())

        topk_acc, head_acc, tail_acc = compute_accuracy(train_loader, model)
        loss_mean = np.mean(loss_history)
        scheduler.step()

        print('Epoch: [{:03d}] \t Loss {:.4f} \t Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(epoch+1, loss_mean, topk_acc[0], head_acc[0], tail_acc[0]))

    torch.save({
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'epoch': epoch},
        osp.join(SAVE_DIR, 'ep{:03d}.pth'.format(EPOCHS))
    )

In [23]:
def test(model, test_loader):
    model.eval()
    topk_acc, head_acc, tail_acc = compute_accuracy(test_loader, model)
    # head : training sample개수가 많은 집함, tail : training samplg이 적은 아이들 -> head의 성능저하를 최소화하고 tail의 성능을 올리는 것이 목표
    print('Acc {:.2f} \t AccHead {:.2f} \t AccTail {:.2f}'.format(topk_acc[0], head_acc[0], tail_acc[0]))

    accuracy = topk_acc[0]
    
    return accuracy

In [24]:
LR = 0.1 # 이거 줄이면 시간 너무오래걸림.
MOMENTUM = 0.9
criterion = nn.CrossEntropyLoss()
torch.manual_seed(0)
train_loader, test_loader, num_classes = get_loaders()

Files already downloaded and verified
Files already downloaded and verified
cls num list(train_dataset):
[4000, 2397, 1437, 861, 516, 309, 185, 111, 66, 40]
cls num list(val_dataset):
[1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]


In [25]:
def train_model(trial):
    cfg = {
        'n_epoch': trial.suggest_categorical('n_epoch', [90, 200]), # 실제 실험에서는 90, 200으로 진행
        'weight_decay': trial.suggest_loguniform('weight_decay', 1e-5, 1e-1),
    }

    model = ResNet18(num_classes)
    model = model.cuda()

    optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=cfg['weight_decay'])
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

    train(cfg['n_epoch'], model, train_loader, criterion, optimizer, scheduler)
    test_accuracy = test(model, test_loader)
    
    return test_accuracy

In [26]:
if __name__ == "__main__":
    sampler = optuna.samplers.TPESampler()
    study = optuna.create_study(sampler=sampler, direction='maximize')
    study.optimize(func=train_model, n_trials=12)
    joblib.dump(study, 'set_10_exp_01.pkl')

[I 2022-06-18 13:16:46,536] A new study created in memory with name: no-name-bbff8b3f-633e-4342-8f76-8d59717dbb55


Epoch: [001] 	 Loss 2.0838 	 Acc 40.32 	 AccHead 43.43 	 AccTail 0.00
Epoch: [002] 	 Loss 1.4879 	 Acc 44.88 	 AccHead 48.33 	 AccTail 0.00
Epoch: [003] 	 Loss 1.3941 	 Acc 40.31 	 AccHead 43.42 	 AccTail 0.00
Epoch: [004] 	 Loss 1.3534 	 Acc 28.84 	 AccHead 31.07 	 AccTail 0.00
Epoch: [005] 	 Loss 1.3696 	 Acc 45.68 	 AccHead 49.19 	 AccTail 0.00
Epoch: [006] 	 Loss 1.3838 	 Acc 45.91 	 AccHead 49.46 	 AccTail 0.00
Epoch: [007] 	 Loss 1.3730 	 Acc 44.14 	 AccHead 47.56 	 AccTail 0.00
Epoch: [008] 	 Loss 1.4172 	 Acc 40.30 	 AccHead 43.42 	 AccTail 0.00
Epoch: [009] 	 Loss 1.3867 	 Acc 25.14 	 AccHead 27.08 	 AccTail 0.00
Epoch: [010] 	 Loss 1.3918 	 Acc 35.01 	 AccHead 37.70 	 AccTail 0.00
Epoch: [011] 	 Loss 1.3805 	 Acc 51.57 	 AccHead 55.55 	 AccTail 0.00
Epoch: [012] 	 Loss 1.3811 	 Acc 42.21 	 AccHead 45.47 	 AccTail 0.00
Epoch: [013] 	 Loss 1.3710 	 Acc 44.46 	 AccHead 47.88 	 AccTail 0.00
Epoch: [014] 	 Loss 1.3725 	 Acc 40.28 	 AccHead 43.40 	 AccTail 0.00
Epoch: [015] 	 Loss 

[I 2022-06-18 13:27:50,089] Trial 0 finished with value: 10.10815715789795 and parameters: {'n_epoch': 90, 'weight_decay': 0.04693209296935963}. Best is trial 0 with value: 10.10815715789795.


Acc 10.11 	 AccHead 20.00 	 AccTail 0.00
Epoch: [001] 	 Loss 2.3162 	 Acc 40.89 	 AccHead 44.05 	 AccTail 0.00
Epoch: [002] 	 Loss 1.3788 	 Acc 47.56 	 AccHead 51.22 	 AccTail 0.00
Epoch: [003] 	 Loss 1.2392 	 Acc 51.57 	 AccHead 55.56 	 AccTail 0.00
Epoch: [004] 	 Loss 1.1589 	 Acc 58.62 	 AccHead 63.13 	 AccTail 0.00
Epoch: [005] 	 Loss 1.1515 	 Acc 46.46 	 AccHead 50.04 	 AccTail 0.00
Epoch: [006] 	 Loss 1.1281 	 Acc 51.29 	 AccHead 55.26 	 AccTail 0.00
Epoch: [007] 	 Loss 1.0885 	 Acc 62.37 	 AccHead 67.17 	 AccTail 0.00
Epoch: [008] 	 Loss 1.1053 	 Acc 59.49 	 AccHead 64.08 	 AccTail 0.00
Epoch: [009] 	 Loss 1.0837 	 Acc 56.45 	 AccHead 60.78 	 AccTail 0.00
Epoch: [010] 	 Loss 1.0820 	 Acc 60.83 	 AccHead 65.50 	 AccTail 0.00
Epoch: [011] 	 Loss 1.0657 	 Acc 59.78 	 AccHead 64.38 	 AccTail 0.00
Epoch: [012] 	 Loss 1.0781 	 Acc 61.07 	 AccHead 65.77 	 AccTail 0.00
Epoch: [013] 	 Loss 1.0741 	 Acc 57.78 	 AccHead 62.26 	 AccTail 0.00
Epoch: [014] 	 Loss 1.0690 	 Acc 57.40 	 AccHead 

[I 2022-06-18 13:52:37,913] Trial 1 finished with value: 23.885574340820312 and parameters: {'n_epoch': 200, 'weight_decay': 0.009352899646879149}. Best is trial 1 with value: 23.885574340820312.


...

Acc 39.22 	 AccHead 61.22 	 AccTail 16.74
Epoch: [001] 	 Loss 2.8888 	 Acc 52.34 	 AccHead 56.39 	 AccTail 0.00
Epoch: [002] 	 Loss 1.5106 	 Acc 56.97 	 AccHead 61.37 	 AccTail 0.00
Epoch: [003] 	 Loss 1.5204 	 Acc 57.24 	 AccHead 61.67 	 AccTail 0.00
Epoch: [004] 	 Loss 1.3587 	 Acc 56.27 	 AccHead 60.64 	 AccTail 0.00
Epoch: [005] 	 Loss 1.2567 	 Acc 60.45 	 AccHead 65.11 	 AccTail 0.00
Epoch: [006] 	 Loss 1.1763 	 Acc 61.37 	 AccHead 66.14 	 AccTail 0.00
Epoch: [007] 	 Loss 1.1914 	 Acc 60.84 	 AccHead 65.51 	 AccTail 0.14
Epoch: [008] 	 Loss 1.1374 	 Acc 65.12 	 AccHead 70.14 	 AccTail 0.14
Epoch: [009] 	 Loss 1.0425 	 Acc 66.97 	 AccHead 72.10 	 AccTail 0.43
Epoch: [010] 	 Loss 1.0192 	 Acc 66.63 	 AccHead 71.75 	 AccTail 0.00
Epoch: [011] 	 Loss 1.0040 	 Acc 67.93 	 AccHead 72.58 	 AccTail 7.52
Epoch: [012] 	 Loss 0.9674 	 Acc 66.56 	 AccHead 71.38 	 AccTail 4.11
Epoch: [013] 	 Loss 0.9675 	 Acc 64.52 	 AccHead 69.50 	 AccTail 0.00
Epoch: [014] 	 Loss 0.9409 	 Acc 69.70 	 AccHead

[I 2022-06-18 15:59:54,050] Trial 10 finished with value: 56.11037826538086 and parameters: {'n_epoch': 200, 'weight_decay': 0.00024735153950860374}. Best is trial 5 with value: 56.24178695678711.


Acc 56.11 	 AccHead 78.78 	 AccTail 32.95
Epoch: [001] 	 Loss 3.0338 	 Acc 46.02 	 AccHead 49.58 	 AccTail 0.00
Epoch: [002] 	 Loss 1.7933 	 Acc 53.47 	 AccHead 57.60 	 AccTail 0.00
Epoch: [003] 	 Loss 1.3907 	 Acc 57.62 	 AccHead 62.08 	 AccTail 0.00
Epoch: [004] 	 Loss 1.3077 	 Acc 58.85 	 AccHead 63.40 	 AccTail 0.00
Epoch: [005] 	 Loss 1.2263 	 Acc 62.50 	 AccHead 67.34 	 AccTail 0.00
Epoch: [006] 	 Loss 1.1493 	 Acc 62.51 	 AccHead 67.33 	 AccTail 0.00
Epoch: [007] 	 Loss 1.0939 	 Acc 63.30 	 AccHead 68.01 	 AccTail 2.13
Epoch: [008] 	 Loss 1.1495 	 Acc 62.20 	 AccHead 66.99 	 AccTail 0.00
Epoch: [009] 	 Loss 1.0518 	 Acc 65.79 	 AccHead 70.13 	 AccTail 9.62
Epoch: [010] 	 Loss 0.9921 	 Acc 67.81 	 AccHead 73.01 	 AccTail 0.14
Epoch: [011] 	 Loss 0.9608 	 Acc 66.23 	 AccHead 71.31 	 AccTail 0.00
Epoch: [012] 	 Loss 0.9523 	 Acc 67.16 	 AccHead 72.27 	 AccTail 1.13
Epoch: [013] 	 Loss 0.9205 	 Acc 69.09 	 AccHead 74.26 	 AccTail 2.40
Epoch: [014] 	 Loss 0.8870 	 Acc 70.46 	 AccHead

[I 2022-06-18 16:24:10,499] Trial 11 finished with value: 54.47285842895508 and parameters: {'n_epoch': 200, 'weight_decay': 0.00022118694089239097}. Best is trial 5 with value: 56.24178695678711.


Acc 54.47 	 AccHead 78.06 	 AccTail 30.37


In [27]:
study.trials

[FrozenTrial(number=0, values=[10.10815715789795], datetime_start=datetime.datetime(2022, 6, 18, 13, 16, 46, 538324), datetime_complete=datetime.datetime(2022, 6, 18, 13, 27, 50, 88341), params={'n_epoch': 90, 'weight_decay': 0.04693209296935963}, distributions={'n_epoch': CategoricalDistribution(choices=(90, 200)), 'weight_decay': LogUniformDistribution(high=0.1, low=1e-05)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=0, state=TrialState.COMPLETE, value=None),
 FrozenTrial(number=1, values=[23.885574340820312], datetime_start=datetime.datetime(2022, 6, 18, 13, 27, 50, 92326), datetime_complete=datetime.datetime(2022, 6, 18, 13, 52, 37, 912480), params={'n_epoch': 200, 'weight_decay': 0.009352899646879149}, distributions={'n_epoch': CategoricalDistribution(choices=(90, 200)), 'weight_decay': LogUniformDistribution(high=0.1, low=1e-05)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=1, state=TrialState.COMPLETE, value=None),
 FrozenTrial(number=2

In [28]:
# !pip install optuna
study = joblib.load('set_10_exp_01.pkl')
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.head(10)

,number,value,duration,params_n_epoch,params_weight_decay
0,0,10.108157,0 days 00:11:03.550017,90,0.046932
1,1,23.885574,0 days 00:24:47.820154,200,0.009353
2,2,50.338623,0 days 00:11:02.469049,90,0.000012
3,3,31.567774,0 days 00:11:04.077658,90,0.002395
4,4,10.108157,0 days 00:11:11.020500,90,0.068901
5,5,56.241787,0 days 00:24:48.703300,200,0.000488
6,6,49.762459,0 days 00:11:06.128622,90,0.000672
7,7,52.198524,0 days 00:11:09.841429,90,0.000042
8,8,30.071768,0 days 00:11:04.912191,90,0.003178
9,9,39.219650,0 days 00:11:17.471789,90,0.001623


In [29]:
# !pip install optuna
study = joblib.load('mnist_optuna.pkl')
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.head(10)

,number,value,duration,params_n_epoch,params_weight_decay
0,0,12.665388,0 days 00:00:14.831324,1,0.000028
1,1,23.866278,0 days 00:00:18.664764,2,0.000070
2,2,15.463085,0 days 00:00:25.052409,3,0.046026


In [12]:
# !pip install optuna
study = joblib.load('set_10_step_01.pkl')
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.head(12)

,number,value,duration,params_n_epoch,params_weight_decay
0,0,35.335155,0 days 00:35:15.375066,200,0.007382
1,1,44.302902,0 days 00:34:58.414487,200,0.000257
2,2,24.699223,0 days 00:35:06.414250,200,0.015852
3,3,42.867252,0 days 00:34:57.689213,200,0.000022
4,4,41.684361,0 days 00:15:56.394683,90,0.000154
5,5,44.646648,0 days 00:35:09.745846,200,0.000288
6,6,41.492268,0 days 00:15:56.506590,90,0.000235
7,7,41.077747,0 days 00:34:48.574028,200,0.002343
8,8,44.211910,0 days 00:35:02.353758,200,0.000421
9,9,38.236782,0 days 00:15:44.087744,90,0.000979


In [16]:
DATASET = 'CIFAR10' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'exp' #['exp', 'step']
IMB_FACTOR = 0.1 #[0.1, 0.01]
train_loader, test_loader, num_classes = get_loaders()

if __name__ == "__main__":
    sampler = optuna.samplers.TPESampler()
    study = optuna.create_study(sampler=sampler, direction='maximize')
    study.optimize(func=train_model, n_trials=10)
    joblib.dump(study, 'set_10_exp_1.pkl')

Files already downloaded and verified
Files already downloaded and verified


[I 2022-06-16 13:43:59,760] A new study created in memory with name: no-name-2dc00495-a06a-42a2-996f-da507754b1c1


cls num list(train_dataset):
[4000, 3097, 2397, 1856, 1437, 1113, 861, 667, 516, 400]
cls num list(val_dataset):
[1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Epoch: [001] 	 Loss 2.5907 	 Acc 36.99 	 AccHead 47.26 	 AccTail 0.00
Epoch: [002] 	 Loss 1.6007 	 Acc 43.44 	 AccHead 55.50 	 AccTail 0.00
Epoch: [003] 	 Loss 1.4940 	 Acc 47.23 	 AccHead 60.36 	 AccTail 0.00
Epoch: [004] 	 Loss 1.4637 	 Acc 43.28 	 AccHead 52.61 	 AccTail 9.65
Epoch: [005] 	 Loss 1.4298 	 Acc 45.63 	 AccHead 57.01 	 AccTail 4.59
Epoch: [006] 	 Loss 1.3918 	 Acc 46.58 	 AccHead 55.32 	 AccTail 15.09
Epoch: [007] 	 Loss 1.3958 	 Acc 45.08 	 AccHead 55.94 	 AccTail 6.02
Epoch: [008] 	 Loss 1.3762 	 Acc 49.43 	 AccHead 58.98 	 AccTail 15.07
Epoch: [009] 	 Loss 1.3572 	 Acc 48.31 	 AccHead 59.56 	 AccTail 7.84
Epoch: [010] 	 Loss 1.3540 	 Acc 38.43 	 AccHead 47.56 	 AccTail 5.60
Epoch: [011] 	 Loss 1.3442 	 Acc 49.77 	 AccHead 59.18 	 AccTail 15.90
Epoch: [012] 	 Loss 1.3365 	 Acc 51.45 	 AccHead 57.2

[I 2022-06-16 13:58:06,218] Trial 0 finished with value: 31.87702178955078 and parameters: {'weight_decay': 0.0056017282604733}. Best is trial 0 with value: 31.87702178955078.


Acc 31.88 	 AccHead 49.50 	 AccTail 13.85
Epoch: [001] 	 Loss 2.4694 	 Acc 36.44 	 AccHead 46.58 	 AccTail 0.00
Epoch: [002] 	 Loss 1.6600 	 Acc 42.75 	 AccHead 54.61 	 AccTail 0.00
Epoch: [003] 	 Loss 1.5249 	 Acc 43.95 	 AccHead 55.71 	 AccTail 1.64
Epoch: [004] 	 Loss 1.4902 	 Acc 48.68 	 AccHead 60.75 	 AccTail 5.13
Epoch: [005] 	 Loss 1.4418 	 Acc 45.60 	 AccHead 51.68 	 AccTail 23.75
Epoch: [006] 	 Loss 1.4002 	 Acc 44.33 	 AccHead 50.19 	 AccTail 23.24
Epoch: [007] 	 Loss 1.3724 	 Acc 41.86 	 AccHead 50.66 	 AccTail 10.11
Epoch: [008] 	 Loss 1.3467 	 Acc 46.80 	 AccHead 53.34 	 AccTail 23.23
Epoch: [009] 	 Loss 1.3384 	 Acc 47.74 	 AccHead 56.19 	 AccTail 17.23
Epoch: [010] 	 Loss 1.3173 	 Acc 51.19 	 AccHead 59.88 	 AccTail 19.92
Epoch: [011] 	 Loss 1.2967 	 Acc 50.55 	 AccHead 56.01 	 AccTail 30.90
Epoch: [012] 	 Loss 1.2884 	 Acc 51.11 	 AccHead 59.81 	 AccTail 19.71
Epoch: [013] 	 Loss 1.3045 	 Acc 49.48 	 AccHead 60.04 	 AccTail 11.46
Epoch: [014] 	 Loss 1.3035 	 Acc 47.98 

...

Acc 55.81 	 AccHead 69.68 	 AccTail 41.61
Epoch: [001] 	 Loss 2.5708 	 Acc 38.80 	 AccHead 49.58 	 AccTail 0.00
Epoch: [002] 	 Loss 1.7285 	 Acc 41.58 	 AccHead 53.13 	 AccTail 0.00
Epoch: [003] 	 Loss 1.6000 	 Acc 43.63 	 AccHead 55.73 	 AccTail 0.03
Epoch: [004] 	 Loss 1.5106 	 Acc 47.39 	 AccHead 59.06 	 AccTail 5.32
Epoch: [005] 	 Loss 1.4675 	 Acc 51.14 	 AccHead 62.83 	 AccTail 9.08
Epoch: [006] 	 Loss 1.4310 	 Acc 50.90 	 AccHead 59.49 	 AccTail 20.03
Epoch: [007] 	 Loss 1.3403 	 Acc 53.54 	 AccHead 60.86 	 AccTail 27.15
Epoch: [008] 	 Loss 1.3062 	 Acc 54.31 	 AccHead 62.80 	 AccTail 23.70
Epoch: [009] 	 Loss 1.2609 	 Acc 56.02 	 AccHead 61.76 	 AccTail 35.30
Epoch: [010] 	 Loss 1.2145 	 Acc 59.93 	 AccHead 70.43 	 AccTail 22.19
Epoch: [011] 	 Loss 1.1648 	 Acc 62.46 	 AccHead 70.93 	 AccTail 31.90
Epoch: [012] 	 Loss 1.0983 	 Acc 63.56 	 AccHead 72.03 	 AccTail 33.09
Epoch: [013] 	 Loss 1.0739 	 Acc 64.89 	 AccHead 72.69 	 AccTail 36.70
Epoch: [014] 	 Loss 1.0326 	 Acc 65.61 	

[I 2022-06-16 15:51:51,521] Trial 8 finished with value: 67.64765167236328 and parameters: {'weight_decay': 2.2929010089254153e-05}. Best is trial 8 with value: 67.64765167236328.


Acc 67.65 	 AccHead 76.40 	 AccTail 58.69
Epoch: [001] 	 Loss 2.3135 	 Acc 43.83 	 AccHead 55.98 	 AccTail 0.03
Epoch: [002] 	 Loss 1.5653 	 Acc 47.15 	 AccHead 58.05 	 AccTail 7.83
Epoch: [003] 	 Loss 1.4773 	 Acc 45.05 	 AccHead 57.01 	 AccTail 2.04
Epoch: [004] 	 Loss 1.4308 	 Acc 48.84 	 AccHead 60.10 	 AccTail 8.29
Epoch: [005] 	 Loss 1.3824 	 Acc 49.42 	 AccHead 60.71 	 AccTail 8.80
Epoch: [006] 	 Loss 1.3526 	 Acc 50.84 	 AccHead 61.95 	 AccTail 10.94
Epoch: [007] 	 Loss 1.3388 	 Acc 46.79 	 AccHead 54.27 	 AccTail 19.85
Epoch: [008] 	 Loss 1.3019 	 Acc 54.00 	 AccHead 64.43 	 AccTail 16.52
Epoch: [009] 	 Loss 1.2770 	 Acc 48.05 	 AccHead 54.08 	 AccTail 26.34
Epoch: [010] 	 Loss 1.2619 	 Acc 52.85 	 AccHead 60.35 	 AccTail 25.75
Epoch: [011] 	 Loss 1.2365 	 Acc 53.41 	 AccHead 61.55 	 AccTail 24.15
Epoch: [012] 	 Loss 1.2503 	 Acc 48.98 	 AccHead 59.80 	 AccTail 10.09
Epoch: [013] 	 Loss 1.2405 	 Acc 55.63 	 AccHead 63.79 	 AccTail 26.31
Epoch: [014] 	 Loss 1.2400 	 Acc 52.97 	

[I 2022-06-16 16:05:54,014] Trial 9 finished with value: 38.612457275390625 and parameters: {'weight_decay': 0.004171034596129968}. Best is trial 8 with value: 67.64765167236328.


Acc 38.61 	 AccHead 54.80 	 AccTail 22.05


In [17]:
study = joblib.load('set_10_exp_1.pkl')
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.head(10)

,number,value,duration,params_weight_decay
0,0,31.877022,0 days 00:14:06.456049,0.005602
1,1,39.987862,0 days 00:14:11.593239,0.004971
2,2,64.199028,0 days 00:14:16.540441,0.000634
3,3,43.112862,0 days 00:14:14.050488,0.003426
4,4,52.740692,0 days 00:14:17.540152,0.002612
5,5,38.723705,0 days 00:14:16.798418,0.004207
6,6,53.772247,0 days 00:14:07.148492,0.002109
7,7,55.805016,0 days 00:14:13.198820,0.001185
8,8,67.647652,0 days 00:14:08.406153,0.000023
9,9,38.612457,0 days 00:14:02.489840,0.004171


In [18]:
DATASET = 'CIFAR10' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'step' #['exp', 'step']
IMB_FACTOR = 0.1 #[0.1, 0.01]
train_loader, test_loader, num_classes = get_loaders()

if __name__ == "__main__":
    sampler = optuna.samplers.TPESampler()
    study = optuna.create_study(sampler=sampler, direction='maximize')
    study.optimize(func=train_model, n_trials=10)
    joblib.dump(study, 'set_10_step_1.pkl')

Files already downloaded and verified
Files already downloaded and verified


[I 2022-06-16 16:06:31,360] A new study created in memory with name: no-name-5257d5c7-9bd6-44ea-86bb-3a68c00d76c8


cls num list(train_dataset):
[4000, 4000, 4000, 4000, 4000, 400, 400, 400, 400, 400]
cls num list(val_dataset):
[1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Epoch: [001] 	 Loss 2.2730 	 Acc 41.02 	 AccHead 45.01 	 AccTail 0.05
Epoch: [002] 	 Loss 1.5382 	 Acc 49.14 	 AccHead 53.92 	 AccTail 0.00
Epoch: [003] 	 Loss 1.3723 	 Acc 53.84 	 AccHead 59.08 	 AccTail 0.00
Epoch: [004] 	 Loss 1.2844 	 Acc 56.14 	 AccHead 61.59 	 AccTail 0.00
Epoch: [005] 	 Loss 1.2165 	 Acc 57.43 	 AccHead 62.79 	 AccTail 2.47
Epoch: [006] 	 Loss 1.1578 	 Acc 61.91 	 AccHead 67.60 	 AccTail 3.31
Epoch: [007] 	 Loss 1.1107 	 Acc 63.77 	 AccHead 69.63 	 AccTail 3.51
Epoch: [008] 	 Loss 1.0437 	 Acc 65.85 	 AccHead 71.59 	 AccTail 6.86
Epoch: [009] 	 Loss 0.9999 	 Acc 68.38 	 AccHead 74.60 	 AccTail 4.44
Epoch: [010] 	 Loss 0.9576 	 Acc 70.20 	 AccHead 76.21 	 AccTail 8.35
Epoch: [011] 	 Loss 0.9262 	 Acc 70.58 	 AccHead 76.26 	 AccTail 12.16
Epoch: [012] 	 Loss 0.8893 	 Acc 72.74 	 AccHead 78.24 	

[I 2022-06-16 16:23:09,457] Trial 0 finished with value: 63.54545593261719 and parameters: {'weight_decay': 0.00027216068742182424}. Best is trial 0 with value: 63.54545593261719.


Acc 63.55 	 AccHead 83.60 	 AccTail 43.08
Epoch: [001] 	 Loss 1.9292 	 Acc 45.01 	 AccHead 49.39 	 AccTail 0.00
Epoch: [002] 	 Loss 1.4164 	 Acc 50.06 	 AccHead 54.94 	 AccTail 0.00
Epoch: [003] 	 Loss 1.3222 	 Acc 54.41 	 AccHead 59.70 	 AccTail 0.00
Epoch: [004] 	 Loss 1.2549 	 Acc 53.57 	 AccHead 58.78 	 AccTail 0.00
Epoch: [005] 	 Loss 1.2041 	 Acc 55.43 	 AccHead 60.79 	 AccTail 0.10
Epoch: [006] 	 Loss 1.1478 	 Acc 59.63 	 AccHead 65.41 	 AccTail 0.21
Epoch: [007] 	 Loss 1.1185 	 Acc 58.46 	 AccHead 64.02 	 AccTail 1.29
Epoch: [008] 	 Loss 1.1204 	 Acc 62.28 	 AccHead 68.08 	 AccTail 2.58
Epoch: [009] 	 Loss 1.0985 	 Acc 59.66 	 AccHead 65.36 	 AccTail 0.98
Epoch: [010] 	 Loss 1.0816 	 Acc 56.31 	 AccHead 61.47 	 AccTail 3.05
Epoch: [011] 	 Loss 1.0829 	 Acc 59.32 	 AccHead 65.00 	 AccTail 0.88
Epoch: [012] 	 Loss 1.0701 	 Acc 54.97 	 AccHead 60.22 	 AccTail 0.88
Epoch: [013] 	 Loss 1.0627 	 Acc 60.31 	 AccHead 66.10 	 AccTail 0.87
Epoch: [014] 	 Loss 1.0498 	 Acc 58.54 	 AccHead

...

Acc 33.07 	 AccHead 61.88 	 AccTail 3.67
Epoch: [001] 	 Loss 2.4242 	 Acc 35.83 	 AccHead 39.30 	 AccTail 0.00
Epoch: [002] 	 Loss 1.5649 	 Acc 47.43 	 AccHead 52.02 	 AccTail 0.00
Epoch: [003] 	 Loss 1.4300 	 Acc 51.32 	 AccHead 56.31 	 AccTail 0.00
Epoch: [004] 	 Loss 1.3596 	 Acc 51.47 	 AccHead 56.47 	 AccTail 0.00
Epoch: [005] 	 Loss 1.3145 	 Acc 54.36 	 AccHead 59.65 	 AccTail 0.00
Epoch: [006] 	 Loss 1.2656 	 Acc 54.47 	 AccHead 59.78 	 AccTail 0.00
Epoch: [007] 	 Loss 1.2200 	 Acc 59.55 	 AccHead 65.32 	 AccTail 0.05
Epoch: [008] 	 Loss 1.1881 	 Acc 56.27 	 AccHead 61.67 	 AccTail 0.77
Epoch: [009] 	 Loss 1.1363 	 Acc 61.68 	 AccHead 67.41 	 AccTail 2.78
Epoch: [010] 	 Loss 1.0999 	 Acc 63.78 	 AccHead 69.91 	 AccTail 0.88
Epoch: [011] 	 Loss 1.0568 	 Acc 64.88 	 AccHead 70.94 	 AccTail 2.63
Epoch: [012] 	 Loss 1.0211 	 Acc 66.31 	 AccHead 72.39 	 AccTail 3.86
Epoch: [013] 	 Loss 0.9959 	 Acc 67.37 	 AccHead 73.47 	 AccTail 4.73
Epoch: [014] 	 Loss 0.9619 	 Acc 69.10 	 AccHead 

[I 2022-06-16 18:36:08,601] Trial 8 finished with value: 62.838382720947266 and parameters: {'weight_decay': 2.1791249080876122e-05}. Best is trial 3 with value: 64.98989868164062.


Acc 62.84 	 AccHead 83.72 	 AccTail 41.53
Epoch: [001] 	 Loss 2.5869 	 Acc 42.08 	 AccHead 46.18 	 AccTail 0.00
Epoch: [002] 	 Loss 1.5668 	 Acc 48.01 	 AccHead 52.67 	 AccTail 0.00
Epoch: [003] 	 Loss 1.4069 	 Acc 51.70 	 AccHead 56.74 	 AccTail 0.00
Epoch: [004] 	 Loss 1.3291 	 Acc 54.50 	 AccHead 59.79 	 AccTail 0.00
Epoch: [005] 	 Loss 1.2735 	 Acc 56.16 	 AccHead 61.61 	 AccTail 0.00
Epoch: [006] 	 Loss 1.2328 	 Acc 58.56 	 AccHead 64.24 	 AccTail 0.10
Epoch: [007] 	 Loss 1.1927 	 Acc 58.83 	 AccHead 64.54 	 AccTail 0.05
Epoch: [008] 	 Loss 1.1558 	 Acc 61.41 	 AccHead 67.33 	 AccTail 0.62
Epoch: [009] 	 Loss 1.1043 	 Acc 62.65 	 AccHead 68.26 	 AccTail 4.90
Epoch: [010] 	 Loss 1.0580 	 Acc 64.39 	 AccHead 69.93 	 AccTail 7.42
Epoch: [011] 	 Loss 1.0231 	 Acc 65.05 	 AccHead 70.87 	 AccTail 5.21
Epoch: [012] 	 Loss 0.9873 	 Acc 69.15 	 AccHead 74.98 	 AccTail 9.14
Epoch: [013] 	 Loss 0.9485 	 Acc 68.61 	 AccHead 74.51 	 AccTail 7.98
Epoch: [014] 	 Loss 0.9121 	 Acc 70.47 	 AccHead

[I 2022-06-16 18:52:53,729] Trial 9 finished with value: 63.272727966308594 and parameters: {'weight_decay': 0.00012069432561766284}. Best is trial 3 with value: 64.98989868164062.


Acc 63.27 	 AccHead 82.48 	 AccTail 43.67


In [19]:
study = joblib.load('set_10_step_1.pkl')
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.head(10)

,number,value,duration,params_weight_decay
0,0,63.545456,0 days 00:16:38.094424,0.000272
1,1,40.272728,0 days 00:16:49.030045,0.002888
2,2,63.303032,0 days 00:16:46.738419,0.000025
3,3,64.989899,0 days 00:16:35.773399,0.000052
4,4,52.121212,0 days 00:16:32.594171,0.000695
5,5,63.828281,0 days 00:16:32.708094,0.000014
6,6,56.181820,0 days 00:16:37.289671,0.000806
7,7,33.070705,0 days 00:16:40.985060,0.003321
8,8,62.838383,0 days 00:16:23.999368,0.000022
9,9,63.272728,0 days 00:16:45.124516,0.000121


In [44]:
def train_model(trial):
    cfg = {
        'n_epoch': trial.suggest_categorical('n_epoch', [90, 200]),
        'weight_decay': trial.suggest_loguniform('weight_decay', 1e-5, 1e-1),
    }

    model = ResNet18(num_classes)
    model = model.cuda()

    optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=cfg['weight_decay'])
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.1)

    train(cfg['n_epoch'], model, train_loader, criterion, optimizer, scheduler)
    test_accuracy = test(model, test_loader)
    
    return test_accuracy

In [45]:
DATASET = 'CIFAR10' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'step' #['exp', 'step']
IMB_FACTOR = 0.01 #[0.1, 0.01]
train_loader, test_loader, num_classes = get_loaders()

if __name__ == "__main__":
    sampler = optuna.samplers.TPESampler()
    study = optuna.create_study(sampler=sampler, direction='maximize')
    study.optimize(func=train_model, n_trials=12)
    joblib.dump(study, 'set_10_step_01.pkl')

Files already downloaded and verified
Files already downloaded and verified


[I 2022-06-17 03:59:53,537] A new study created in memory with name: no-name-1ab214c3-ed24-4b54-aa4c-479bd490f2bf


cls num list(train_dataset):
[4000, 4000, 4000, 4000, 4000, 40, 40, 40, 40, 40]
cls num list(val_dataset):
[1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]
Epoch: [001] 	 Loss 1.8358 	 Acc 46.34 	 AccHead 46.73 	 AccTail 0.00
Epoch: [002] 	 Loss 1.1913 	 Acc 50.63 	 AccHead 51.05 	 AccTail 0.00
Epoch: [003] 	 Loss 1.1232 	 Acc 53.39 	 AccHead 53.84 	 AccTail 0.00
Epoch: [004] 	 Loss 1.1030 	 Acc 52.93 	 AccHead 53.37 	 AccTail 0.00
Epoch: [005] 	 Loss 1.0955 	 Acc 48.13 	 AccHead 48.53 	 AccTail 0.00
Epoch: [006] 	 Loss 1.0986 	 Acc 52.58 	 AccHead 53.02 	 AccTail 0.00
Epoch: [007] 	 Loss 1.0850 	 Acc 55.49 	 AccHead 55.96 	 AccTail 0.00
Epoch: [008] 	 Loss 1.0658 	 Acc 54.37 	 AccHead 54.83 	 AccTail 0.00
Epoch: [009] 	 Loss 1.0568 	 Acc 45.61 	 AccHead 45.99 	 AccTail 0.00
Epoch: [010] 	 Loss 1.0612 	 Acc 48.83 	 AccHead 49.23 	 AccTail 0.00
Epoch: [011] 	 Loss 1.0593 	 Acc 53.61 	 AccHead 54.06 	 AccTail 0.00
Epoch: [012] 	 Loss 1.0414 	 Acc 59.57 	 AccHead 60.07 	 AccTa

[I 2022-06-17 04:35:08,914] Trial 0 finished with value: 35.33515548706055 and parameters: {'n_epoch': 200, 'weight_decay': 0.007381922467256987}. Best is trial 0 with value: 35.33515548706055.


Acc 35.34 	 AccHead 69.90 	 AccTail 0.00
Epoch: [001] 	 Loss 2.2022 	 Acc 37.07 	 AccHead 37.38 	 AccTail 0.00
Epoch: [002] 	 Loss 1.3420 	 Acc 53.99 	 AccHead 54.44 	 AccTail 0.00
Epoch: [003] 	 Loss 1.1420 	 Acc 56.05 	 AccHead 56.51 	 AccTail 0.00
Epoch: [004] 	 Loss 1.0641 	 Acc 59.01 	 AccHead 59.50 	 AccTail 0.00
Epoch: [005] 	 Loss 1.0092 	 Acc 63.43 	 AccHead 63.96 	 AccTail 0.00
Epoch: [006] 	 Loss 0.9587 	 Acc 62.31 	 AccHead 62.83 	 AccTail 0.00
Epoch: [007] 	 Loss 0.9082 	 Acc 68.30 	 AccHead 68.87 	 AccTail 0.00
Epoch: [008] 	 Loss 0.8511 	 Acc 69.25 	 AccHead 69.83 	 AccTail 0.00
Epoch: [009] 	 Loss 0.8218 	 Acc 72.76 	 AccHead 73.37 	 AccTail 0.00
Epoch: [010] 	 Loss 0.7763 	 Acc 72.69 	 AccHead 73.30 	 AccTail 0.00
Epoch: [011] 	 Loss 0.7442 	 Acc 74.85 	 AccHead 75.48 	 AccTail 0.00
Epoch: [012] 	 Loss 0.7259 	 Acc 71.86 	 AccHead 72.46 	 AccTail 0.00
Epoch: [013] 	 Loss 0.6984 	 Acc 75.30 	 AccHead 75.93 	 AccTail 0.00
Epoch: [014] 	 Loss 0.6696 	 Acc 75.66 	 AccHead 

...

Acc 38.24 	 AccHead 75.64 	 AccTail 0.00
Epoch: [001] 	 Loss 1.7342 	 Acc 27.69 	 AccHead 27.92 	 AccTail 0.00
Epoch: [002] 	 Loss 1.5466 	 Acc 19.71 	 AccHead 19.88 	 AccTail 0.00
Epoch: [003] 	 Loss 1.5703 	 Acc 19.84 	 AccHead 20.01 	 AccTail 0.00
Epoch: [004] 	 Loss 1.5483 	 Acc 23.27 	 AccHead 23.47 	 AccTail 0.00
Epoch: [005] 	 Loss 1.5847 	 Acc 19.84 	 AccHead 20.01 	 AccTail 0.00
Epoch: [006] 	 Loss 1.5899 	 Acc 19.82 	 AccHead 19.98 	 AccTail 0.00
Epoch: [007] 	 Loss 1.6144 	 Acc 19.84 	 AccHead 20.01 	 AccTail 0.00
Epoch: [008] 	 Loss 1.6514 	 Acc 19.86 	 AccHead 20.03 	 AccTail 0.00
Epoch: [009] 	 Loss 1.6539 	 Acc 19.82 	 AccHead 19.99 	 AccTail 0.00
Epoch: [010] 	 Loss 1.7305 	 Acc 19.85 	 AccHead 20.02 	 AccTail 0.00
Epoch: [011] 	 Loss 1.7311 	 Acc 19.81 	 AccHead 19.98 	 AccTail 0.00
Epoch: [012] 	 Loss 1.7334 	 Acc 19.82 	 AccHead 19.98 	 AccTail 0.00
Epoch: [013] 	 Loss 1.7354 	 Acc 19.84 	 AccHead 20.01 	 AccTail 0.00
Epoch: [014] 	 Loss 1.7341 	 Acc 19.83 	 AccHead 

[I 2022-06-17 09:28:00,041] Trial 10 finished with value: 10.110200881958008 and parameters: {'n_epoch': 200, 'weight_decay': 0.08451228520498819}. Best is trial 5 with value: 44.64664840698242.


Acc 10.11 	 AccHead 20.00 	 AccTail 0.00
Epoch: [001] 	 Loss 2.1324 	 Acc 48.99 	 AccHead 49.40 	 AccTail 0.00
Epoch: [002] 	 Loss 1.4165 	 Acc 52.67 	 AccHead 53.11 	 AccTail 0.00
Epoch: [003] 	 Loss 1.1909 	 Acc 56.27 	 AccHead 56.74 	 AccTail 0.00
Epoch: [004] 	 Loss 1.1202 	 Acc 51.12 	 AccHead 51.55 	 AccTail 0.00
Epoch: [005] 	 Loss 1.0559 	 Acc 55.81 	 AccHead 56.28 	 AccTail 0.00
Epoch: [006] 	 Loss 1.0189 	 Acc 63.68 	 AccHead 64.22 	 AccTail 0.00
Epoch: [007] 	 Loss 0.9492 	 Acc 65.56 	 AccHead 66.11 	 AccTail 0.00
Epoch: [008] 	 Loss 0.9123 	 Acc 67.53 	 AccHead 68.09 	 AccTail 0.00
Epoch: [009] 	 Loss 0.8742 	 Acc 66.50 	 AccHead 67.05 	 AccTail 0.00
Epoch: [010] 	 Loss 0.8273 	 Acc 68.97 	 AccHead 69.54 	 AccTail 0.00
Epoch: [011] 	 Loss 0.7817 	 Acc 72.39 	 AccHead 72.99 	 AccTail 0.00
Epoch: [012] 	 Loss 0.7559 	 Acc 69.58 	 AccHead 70.16 	 AccTail 0.00
Epoch: [013] 	 Loss 0.7289 	 Acc 75.95 	 AccHead 76.58 	 AccTail 0.00
Epoch: [014] 	 Loss 0.6914 	 Acc 74.76 	 AccHead 

[I 2022-06-17 10:03:08,031] Trial 11 finished with value: 43.49408721923828 and parameters: {'n_epoch': 200, 'weight_decay': 3.566967303889489e-05}. Best is trial 5 with value: 44.64664840698242.


Acc 43.49 	 AccHead 84.76 	 AccTail 1.31


In [46]:
study = joblib.load('set_10_step_01.pkl')
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.head(10)

,number,value,duration,params_n_epoch,params_weight_decay
0,0,35.335155,0 days 00:35:15.375066,200,0.007382
1,1,44.302902,0 days 00:34:58.414487,200,0.000257
2,2,24.699223,0 days 00:35:06.414250,200,0.015852
3,3,42.867252,0 days 00:34:57.689213,200,0.000022
4,4,41.684361,0 days 00:15:56.394683,90,0.000154
5,5,44.646648,0 days 00:35:09.745846,200,0.000288
6,6,41.492268,0 days 00:15:56.506590,90,0.000235
7,7,41.077747,0 days 00:34:48.574028,200,0.002343
8,8,44.211910,0 days 00:35:02.353758,200,0.000421
9,9,38.236782,0 days 00:15:44.087744,90,0.000979


In [47]:
DATASET = 'CIFAR100' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'exp' #['exp', 'step']
IMB_FACTOR = 0.1 #[0.1, 0.01]
train_loader, test_loader, num_classes = get_loaders()

if __name__ == "__main__":
    sampler = optuna.samplers.TPESampler()
    study = optuna.create_study(sampler=sampler, direction='maximize')
    study.optimize(func=train_model, n_trials=12)
    joblib.dump(study, 'set_100_exp_1.pkl')

Files already downloaded and verified
Files already downloaded and verified


[I 2022-06-17 10:03:45,637] A new study created in memory with name: no-name-12208c39-adbe-4b0d-a0b0-ce6e64049729


cls num list(train_dataset):
[400, 390, 381, 373, 364, 356, 347, 339, 332, 324, 316, 309, 302, 295, 288, 282, 275, 269, 263, 257, 251, 245, 239, 234, 228, 223, 218, 213, 208, 203, 199, 194, 190, 185, 181, 177, 173, 169, 165, 161, 157, 154, 150, 147, 143, 140, 137, 134, 130, 127, 125, 122, 119, 116, 113, 111, 108, 106, 103, 101, 99, 96, 94, 92, 90, 88, 86, 84, 82, 80, 78, 76, 74, 73, 71, 69, 68, 66, 65, 63, 62, 60, 59, 58, 56, 55, 54, 52, 51, 50, 49, 48, 47, 45, 44, 43, 42, 41, 40, 40]
cls num list(val_dataset):
[100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 10

[I 2022-06-17 10:34:46,028] Trial 0 finished with value: 1.033484935760498 and parameters: {'n_epoch': 200, 'weight_decay': 0.037809461247346354}. Best is trial 0 with value: 1.033484935760498.


Acc 1.03 	 AccHead 2.08 	 AccTail 0.00
Epoch: [001] 	 Loss 4.5252 	 Acc 8.66 	 AccHead 10.98 	 AccTail 1.21
Epoch: [002] 	 Loss 3.8960 	 Acc 13.99 	 AccHead 16.83 	 AccTail 4.83
Epoch: [003] 	 Loss 3.6699 	 Acc 15.16 	 AccHead 19.31 	 AccTail 1.76
Epoch: [004] 	 Loss 3.4873 	 Acc 18.99 	 AccHead 22.56 	 AccTail 7.48
Epoch: [005] 	 Loss 3.3559 	 Acc 22.41 	 AccHead 27.19 	 AccTail 7.02
Epoch: [006] 	 Loss 3.2266 	 Acc 23.74 	 AccHead 28.25 	 AccTail 9.21
Epoch: [007] 	 Loss 3.1152 	 Acc 25.83 	 AccHead 29.51 	 AccTail 13.96
Epoch: [008] 	 Loss 3.0290 	 Acc 28.98 	 AccHead 34.35 	 AccTail 11.69
Epoch: [009] 	 Loss 2.9343 	 Acc 28.11 	 AccHead 32.41 	 AccTail 14.29
Epoch: [010] 	 Loss 2.8684 	 Acc 26.05 	 AccHead 30.67 	 AccTail 11.16
Epoch: [011] 	 Loss 2.7854 	 Acc 33.47 	 AccHead 39.25 	 AccTail 14.86
Epoch: [012] 	 Loss 2.6962 	 Acc 32.82 	 AccHead 37.68 	 AccTail 17.15
Epoch: [013] 	 Loss 2.6257 	 Acc 33.72 	 AccHead 38.65 	 AccTail 17.81
Epoch: [014] 	 Loss 2.5832 	 Acc 35.80 	 AccH

...

Acc 1.03 	 AccHead 2.08 	 AccTail 0.00
Epoch: [001] 	 Loss 4.4310 	 Acc 7.08 	 AccHead 9.27 	 AccTail 0.00
Epoch: [002] 	 Loss 3.8907 	 Acc 9.80 	 AccHead 12.49 	 AccTail 1.16
Epoch: [003] 	 Loss 3.7296 	 Acc 14.23 	 AccHead 18.05 	 AccTail 1.92
Epoch: [004] 	 Loss 3.5949 	 Acc 13.70 	 AccHead 16.90 	 AccTail 3.40
Epoch: [005] 	 Loss 3.5274 	 Acc 16.57 	 AccHead 20.31 	 AccTail 4.47
Epoch: [006] 	 Loss 3.4742 	 Acc 15.36 	 AccHead 18.24 	 AccTail 6.08
Epoch: [007] 	 Loss 3.4299 	 Acc 16.66 	 AccHead 20.87 	 AccTail 3.11
Epoch: [008] 	 Loss 3.3988 	 Acc 17.61 	 AccHead 21.56 	 AccTail 4.89
Epoch: [009] 	 Loss 3.3701 	 Acc 14.95 	 AccHead 17.63 	 AccTail 6.30
Epoch: [010] 	 Loss 3.3081 	 Acc 21.30 	 AccHead 25.75 	 AccTail 6.97
Epoch: [011] 	 Loss 3.2978 	 Acc 19.12 	 AccHead 22.56 	 AccTail 8.07
Epoch: [012] 	 Loss 3.2578 	 Acc 21.04 	 AccHead 24.34 	 AccTail 10.40
Epoch: [013] 	 Loss 3.2281 	 Acc 18.02 	 AccHead 21.23 	 AccTail 7.70
Epoch: [014] 	 Loss 3.2037 	 Acc 19.95 	 AccHead 23.2

[I 2022-06-17 13:58:48,934] Trial 10 finished with value: 9.394377708435059 and parameters: {'n_epoch': 200, 'weight_decay': 0.003728674914045116}. Best is trial 6 with value: 10.572550773620605.


Acc 9.39 	 AccHead 18.67 	 AccTail 0.27
Epoch: [001] 	 Loss 4.4140 	 Acc 6.04 	 AccHead 7.90 	 AccTail 0.03
Epoch: [002] 	 Loss 3.8572 	 Acc 10.98 	 AccHead 14.37 	 AccTail 0.03
Epoch: [003] 	 Loss 3.7204 	 Acc 13.85 	 AccHead 17.28 	 AccTail 2.79
Epoch: [004] 	 Loss 3.6362 	 Acc 12.48 	 AccHead 15.80 	 AccTail 1.81
Epoch: [005] 	 Loss 3.5678 	 Acc 16.06 	 AccHead 19.79 	 AccTail 4.03
Epoch: [006] 	 Loss 3.5016 	 Acc 15.18 	 AccHead 18.09 	 AccTail 5.81
Epoch: [007] 	 Loss 3.4561 	 Acc 16.49 	 AccHead 19.75 	 AccTail 6.00
Epoch: [008] 	 Loss 3.4502 	 Acc 18.69 	 AccHead 23.51 	 AccTail 3.16
Epoch: [009] 	 Loss 3.3886 	 Acc 16.41 	 AccHead 19.12 	 AccTail 7.70
Epoch: [010] 	 Loss 3.3644 	 Acc 16.28 	 AccHead 19.84 	 AccTail 4.83
Epoch: [011] 	 Loss 3.3679 	 Acc 17.85 	 AccHead 21.87 	 AccTail 4.89
Epoch: [012] 	 Loss 3.3404 	 Acc 18.36 	 AccHead 21.79 	 AccTail 7.32
Epoch: [013] 	 Loss 3.3113 	 Acc 16.50 	 AccHead 20.25 	 AccTail 4.43
Epoch: [014] 	 Loss 3.2895 	 Acc 20.59 	 AccHead 24.

[I 2022-06-17 14:28:40,257] Trial 11 finished with value: 9.487391471862793 and parameters: {'n_epoch': 200, 'weight_decay': 0.004207112562455028}. Best is trial 6 with value: 10.572550773620605.


Acc 9.49 	 AccHead 18.94 	 AccTail 0.18


In [48]:
study = joblib.load('set_100_exp_1.pkl')
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.head(10)

,number,value,duration,params_n_epoch,params_weight_decay
0,0,1.033485,0 days 00:31:00.387800,200,0.037809
1,1,8.340223,0 days 00:13:55.552840,90,0.000682
2,2,9.270360,0 days 00:30:53.134421,200,0.000013
3,3,8.691608,0 days 00:14:00.592525,90,0.000073
4,4,9.125672,0 days 00:13:54.098197,90,0.000073
5,5,10.272840,0 days 00:30:27.863838,200,0.000257
6,6,10.572551,0 days 00:30:33.004942,200,0.001405
7,7,8.360893,0 days 00:13:42.935642,90,0.000526
8,8,1.033485,0 days 00:13:18.556360,90,0.068770
9,9,1.033485,0 days 00:13:26.297676,90,0.047246


In [50]:
DATASET = 'CIFAR100' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'exp' #['exp', 'step']
IMB_FACTOR = 0.01 #[0.1, 0.01]
train_loader, test_loader, num_classes = get_loaders()

if __name__ == "__main__":
    sampler = optuna.samplers.TPESampler()
    study = optuna.create_study(sampler=sampler, direction='maximize')
    study.optimize(func=train_model, n_trials=12)
    joblib.dump(study, 'set_100_exp_01.pkl')

Files already downloaded and verified
Files already downloaded and verified


[I 2022-06-17 16:03:15,090] A new study created in memory with name: no-name-321e1921-bd7e-447f-b691-f731909c9dd7


cls num list(train_dataset):
[400, 381, 364, 347, 332, 316, 302, 288, 275, 263, 251, 239, 228, 218, 208, 199, 190, 181, 173, 165, 157, 150, 143, 137, 130, 125, 119, 113, 108, 103, 99, 94, 90, 86, 82, 78, 74, 71, 68, 65, 62, 59, 56, 54, 51, 49, 47, 44, 42, 40, 39, 37, 35, 33, 32, 30, 29, 28, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 16, 15, 14, 14, 13, 12, 12, 11, 11, 10, 10, 9, 9, 8, 8, 8, 7, 7, 6, 6, 6, 6, 5, 5, 5, 5, 4, 4, 4, 4, 4]
cls num list(val_dataset):
[100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]
Epoch: [001] 	 Loss 4.2883 	 Acc

[I 2022-06-17 16:13:36,390] Trial 0 finished with value: 1.1318795680999756 and parameters: {'n_epoch': 90, 'weight_decay': 0.029962751900251588}. Best is trial 0 with value: 1.1318795680999756.


Acc 1.13 	 AccHead 2.27 	 AccTail 0.00
Epoch: [001] 	 Loss 4.4172 	 Acc 13.77 	 AccHead 15.08 	 AccTail 0.00
Epoch: [002] 	 Loss 3.7580 	 Acc 13.76 	 AccHead 15.06 	 AccTail 0.00
Epoch: [003] 	 Loss 3.5137 	 Acc 18.62 	 AccHead 20.38 	 AccTail 0.00
Epoch: [004] 	 Loss 3.3362 	 Acc 20.95 	 AccHead 22.82 	 AccTail 1.47
Epoch: [005] 	 Loss 3.2219 	 Acc 21.62 	 AccHead 23.44 	 AccTail 2.42
Epoch: [006] 	 Loss 3.0980 	 Acc 23.30 	 AccHead 25.30 	 AccTail 2.40
Epoch: [007] 	 Loss 2.9979 	 Acc 26.67 	 AccHead 28.61 	 AccTail 6.20
Epoch: [008] 	 Loss 2.9962 	 Acc 28.04 	 AccHead 30.24 	 AccTail 4.85
Epoch: [009] 	 Loss 2.8349 	 Acc 28.25 	 AccHead 30.49 	 AccTail 4.70
Epoch: [010] 	 Loss 2.7655 	 Acc 30.18 	 AccHead 32.65 	 AccTail 4.17
Epoch: [011] 	 Loss 2.6749 	 Acc 30.74 	 AccHead 33.19 	 AccTail 4.74
Epoch: [012] 	 Loss 2.5877 	 Acc 33.23 	 AccHead 35.82 	 AccTail 5.92
Epoch: [013] 	 Loss 2.5619 	 Acc 35.94 	 AccHead 38.87 	 AccTail 5.21
Epoch: [014] 	 Loss 2.4583 	 Acc 37.26 	 AccHead 39

...

Acc 5.91 	 AccHead 11.83 	 AccTail 0.02
Epoch: [001] 	 Loss 4.5958 	 Acc 13.46 	 AccHead 14.73 	 AccTail 0.00
Epoch: [002] 	 Loss 3.7850 	 Acc 15.65 	 AccHead 17.12 	 AccTail 0.00
Epoch: [003] 	 Loss 3.5140 	 Acc 18.39 	 AccHead 20.13 	 AccTail 0.00
Epoch: [004] 	 Loss 3.3308 	 Acc 20.57 	 AccHead 22.40 	 AccTail 1.22
Epoch: [005] 	 Loss 3.1809 	 Acc 24.63 	 AccHead 26.70 	 AccTail 2.70
Epoch: [006] 	 Loss 3.0882 	 Acc 25.02 	 AccHead 27.18 	 AccTail 2.41
Epoch: [007] 	 Loss 2.9970 	 Acc 26.67 	 AccHead 28.96 	 AccTail 2.43
Epoch: [008] 	 Loss 2.9138 	 Acc 29.99 	 AccHead 31.97 	 AccTail 9.14
Epoch: [009] 	 Loss 2.8128 	 Acc 31.65 	 AccHead 33.84 	 AccTail 8.48
Epoch: [010] 	 Loss 2.7306 	 Acc 32.59 	 AccHead 35.06 	 AccTail 6.81
Epoch: [011] 	 Loss 2.6504 	 Acc 32.91 	 AccHead 35.22 	 AccTail 8.39
Epoch: [012] 	 Loss 2.5759 	 Acc 34.70 	 AccHead 37.02 	 AccTail 10.43
Epoch: [013] 	 Loss 2.5045 	 Acc 37.59 	 AccHead 40.11 	 AccTail 11.04
Epoch: [014] 	 Loss 2.4273 	 Acc 39.56 	 AccHead

[I 2022-06-17 19:03:42,494] Trial 10 finished with value: 5.233644962310791 and parameters: {'n_epoch': 200, 'weight_decay': 1.9900147700126848e-05}. Best is trial 1 with value: 6.085150718688965.


Acc 5.23 	 AccHead 10.35 	 AccTail 0.14
Epoch: [001] 	 Loss 4.4502 	 Acc 11.95 	 AccHead 13.00 	 AccTail 0.81
Epoch: [002] 	 Loss 3.7310 	 Acc 16.66 	 AccHead 18.17 	 AccTail 0.81
Epoch: [003] 	 Loss 3.4609 	 Acc 19.80 	 AccHead 21.67 	 AccTail 0.00
Epoch: [004] 	 Loss 3.3011 	 Acc 20.81 	 AccHead 22.51 	 AccTail 3.08
Epoch: [005] 	 Loss 3.1876 	 Acc 23.67 	 AccHead 25.55 	 AccTail 3.90
Epoch: [006] 	 Loss 3.0444 	 Acc 26.74 	 AccHead 29.01 	 AccTail 2.82
Epoch: [007] 	 Loss 2.9403 	 Acc 27.61 	 AccHead 29.98 	 AccTail 2.69
Epoch: [008] 	 Loss 2.8422 	 Acc 29.23 	 AccHead 31.65 	 AccTail 3.89
Epoch: [009] 	 Loss 2.7711 	 Acc 32.78 	 AccHead 35.32 	 AccTail 5.81
Epoch: [010] 	 Loss 2.6837 	 Acc 32.33 	 AccHead 34.61 	 AccTail 8.36
Epoch: [011] 	 Loss 2.5790 	 Acc 36.45 	 AccHead 39.46 	 AccTail 4.83
Epoch: [012] 	 Loss 2.5239 	 Acc 37.40 	 AccHead 39.58 	 AccTail 14.57
Epoch: [013] 	 Loss 2.4239 	 Acc 39.60 	 AccHead 42.45 	 AccTail 9.46
Epoch: [014] 	 Loss 2.3681 	 Acc 38.85 	 AccHead 

[I 2022-06-17 19:27:31,273] Trial 11 finished with value: 5.763239860534668 and parameters: {'n_epoch': 200, 'weight_decay': 0.00010004247095947701}. Best is trial 1 with value: 6.085150718688965.


Acc 5.76 	 AccHead 11.35 	 AccTail 0.21


In [51]:
study = joblib.load('set_100_exp_01.pkl')
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.head(10)

,number,value,duration,params_n_epoch,params_weight_decay
0,0,1.131880,0 days 00:10:21.297956,90,0.029963
1,1,6.085151,0 days 00:23:31.749703,200,0.001043
2,2,5.192108,0 days 00:10:51.839308,90,0.000493
3,3,5.430945,0 days 00:10:51.158423,90,0.000039
4,4,1.038422,0 days 00:23:44.966203,200,0.066359
5,5,6.085151,0 days 00:23:45.929806,200,0.000571
6,6,2.253375,0 days 00:10:17.861273,90,0.022016
7,7,2.585670,0 days 00:10:29.631700,90,0.007837
8,8,4.953271,0 days 00:10:33.087734,90,0.001553
9,9,5.908619,0 days 00:22:57.850075,200,0.000697


In [52]:
DATASET = 'CIFAR100' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'step' #['exp', 'step']
IMB_FACTOR = 0.1 #[0.1, 0.01]
train_loader, test_loader, num_classes = get_loaders()

if __name__ == "__main__":
    sampler = optuna.samplers.TPESampler()
    study = optuna.create_study(sampler=sampler, direction='maximize')
    study.optimize(func=train_model, n_trials=12)
    joblib.dump(study, 'set_100_step_1.pkl')

Files already downloaded and verified
Files already downloaded and verified


[I 2022-06-17 19:28:10,867] A new study created in memory with name: no-name-c7df62e4-26df-4cb5-bfbd-4aaeb4fa50e4


cls num list(train_dataset):
[400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40]
cls num list(val_dataset):
[100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 10

[I 2022-06-17 20:04:36,708] Trial 0 finished with value: 9.79477596282959 and parameters: {'n_epoch': 200, 'weight_decay': 0.0023139210554174432}. Best is trial 0 with value: 9.79477596282959.


Acc 9.79 	 AccHead 17.21 	 AccTail 2.45
Epoch: [001] 	 Loss 4.1851 	 Acc 5.85 	 AccHead 6.65 	 AccTail 0.00
Epoch: [002] 	 Loss 3.7992 	 Acc 9.88 	 AccHead 11.24 	 AccTail 0.00
Epoch: [003] 	 Loss 3.6958 	 Acc 9.07 	 AccHead 9.18 	 AccTail 8.28
Epoch: [004] 	 Loss 3.6457 	 Acc 13.79 	 AccHead 14.83 	 AccTail 6.22
Epoch: [005] 	 Loss 3.5874 	 Acc 13.26 	 AccHead 14.19 	 AccTail 6.45
Epoch: [006] 	 Loss 3.5562 	 Acc 13.22 	 AccHead 14.14 	 AccTail 6.51
Epoch: [007] 	 Loss 3.5398 	 Acc 13.18 	 AccHead 14.27 	 AccTail 5.25
Epoch: [008] 	 Loss 3.5208 	 Acc 12.63 	 AccHead 13.77 	 AccTail 4.38
Epoch: [009] 	 Loss 3.5156 	 Acc 13.69 	 AccHead 15.39 	 AccTail 1.37
Epoch: [010] 	 Loss 3.4999 	 Acc 13.78 	 AccHead 14.83 	 AccTail 6.13
Epoch: [011] 	 Loss 3.4906 	 Acc 14.79 	 AccHead 16.27 	 AccTail 4.00
Epoch: [012] 	 Loss 3.4827 	 Acc 11.48 	 AccHead 12.49 	 AccTail 4.11
Epoch: [013] 	 Loss 3.4736 	 Acc 12.87 	 AccHead 13.51 	 AccTail 8.18
Epoch: [014] 	 Loss 3.4693 	 Acc 10.74 	 AccHead 11.86 

...

Acc 8.67 	 AccHead 15.15 	 AccTail 2.25
Epoch: [001] 	 Loss 4.2464 	 Acc 10.14 	 AccHead 10.97 	 AccTail 4.07
Epoch: [002] 	 Loss 3.7374 	 Acc 13.20 	 AccHead 14.97 	 AccTail 0.30
Epoch: [003] 	 Loss 3.5416 	 Acc 15.48 	 AccHead 16.84 	 AccTail 5.60
Epoch: [004] 	 Loss 3.4074 	 Acc 18.43 	 AccHead 20.20 	 AccTail 5.55
Epoch: [005] 	 Loss 3.2564 	 Acc 22.84 	 AccHead 24.89 	 AccTail 7.92
Epoch: [006] 	 Loss 3.1027 	 Acc 23.18 	 AccHead 25.22 	 AccTail 8.34
Epoch: [007] 	 Loss 2.9744 	 Acc 25.88 	 AccHead 28.00 	 AccTail 10.45
Epoch: [008] 	 Loss 2.8717 	 Acc 28.42 	 AccHead 30.70 	 AccTail 11.76
Epoch: [009] 	 Loss 2.7757 	 Acc 28.67 	 AccHead 30.87 	 AccTail 12.64
Epoch: [010] 	 Loss 2.6887 	 Acc 32.86 	 AccHead 35.58 	 AccTail 13.07
Epoch: [011] 	 Loss 2.6121 	 Acc 32.20 	 AccHead 35.18 	 AccTail 10.50
Epoch: [012] 	 Loss 2.5455 	 Acc 34.94 	 AccHead 37.60 	 AccTail 15.58
Epoch: [013] 	 Loss 2.4736 	 Acc 35.50 	 AccHead 38.23 	 AccTail 15.66
Epoch: [014] 	 Loss 2.4098 	 Acc 37.01 	 Ac

[I 2022-06-18 00:31:05,343] Trial 10 finished with value: 9.670397758483887 and parameters: {'n_epoch': 200, 'weight_decay': 0.00021123599820854373}. Best is trial 5 with value: 10.157546043395996.


Acc 9.67 	 AccHead 16.81 	 AccTail 2.60
Epoch: [001] 	 Loss 4.2221 	 Acc 10.03 	 AccHead 11.06 	 AccTail 2.56
Epoch: [002] 	 Loss 3.7014 	 Acc 13.75 	 AccHead 15.03 	 AccTail 4.41
Epoch: [003] 	 Loss 3.5060 	 Acc 15.57 	 AccHead 17.07 	 AccTail 4.68
Epoch: [004] 	 Loss 3.3602 	 Acc 17.91 	 AccHead 19.11 	 AccTail 9.20
Epoch: [005] 	 Loss 3.2443 	 Acc 21.97 	 AccHead 23.89 	 AccTail 8.00
Epoch: [006] 	 Loss 3.1303 	 Acc 24.72 	 AccHead 27.12 	 AccTail 7.22
Epoch: [007] 	 Loss 3.0099 	 Acc 25.32 	 AccHead 28.19 	 AccTail 4.42
Epoch: [008] 	 Loss 2.9230 	 Acc 26.57 	 AccHead 28.68 	 AccTail 11.18
Epoch: [009] 	 Loss 2.8203 	 Acc 28.13 	 AccHead 30.80 	 AccTail 8.72
Epoch: [010] 	 Loss 2.7436 	 Acc 31.14 	 AccHead 34.02 	 AccTail 10.20
Epoch: [011] 	 Loss 2.6732 	 Acc 31.85 	 AccHead 34.59 	 AccTail 11.91
Epoch: [012] 	 Loss 2.6036 	 Acc 32.48 	 AccHead 35.26 	 AccTail 12.22
Epoch: [013] 	 Loss 2.5405 	 Acc 34.31 	 AccHead 37.57 	 AccTail 10.55
Epoch: [014] 	 Loss 2.5006 	 Acc 34.42 	 AccH

[I 2022-06-18 01:07:59,681] Trial 11 finished with value: 10.199005126953125 and parameters: {'n_epoch': 200, 'weight_decay': 0.00047918568755358793}. Best is trial 11 with value: 10.199005126953125.


Acc 10.20 	 AccHead 17.92 	 AccTail 2.56


In [53]:
study = joblib.load('set_100_step_1.pkl')
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.head(10)

,number,value,duration,params_n_epoch,params_weight_decay
0,0,9.794776,0 days 00:36:25.838325,200,0.002314
1,1,5.358624,0 days 00:36:40.001090,200,0.005399
2,2,3.430763,0 days 00:16:31.675795,90,0.003465
3,3,1.036484,0 days 00:16:42.299458,90,0.037093
4,4,1.036484,0 days 00:36:49.319264,200,0.019981
5,5,10.157546,0 days 00:36:59.615525,200,0.001366
6,6,1.886401,0 days 00:16:33.863514,90,0.013638
7,7,1.036484,0 days 00:36:39.593646,200,0.022443
8,8,8.665009,0 days 00:16:31.285765,90,0.000019
9,9,8.665009,0 days 00:16:27.817277,90,0.000015


In [54]:
DATASET = 'CIFAR100' #['CIFAR10', 'CIFAR100']
IMB_TYPE = 'step' #['exp', 'step']
IMB_FACTOR = 0.01 #[0.1, 0.01]
train_loader, test_loader, num_classes = get_loaders()

if __name__ == "__main__":
    sampler = optuna.samplers.TPESampler()
    study = optuna.create_study(sampler=sampler, direction='maximize')
    study.optimize(func=train_model, n_trials=12)
    joblib.dump(study, 'set_100_step_01.pkl')

Files already downloaded and verified
Files already downloaded and verified


[I 2022-06-18 01:08:37,432] A new study created in memory with name: no-name-c9c92fae-1063-47cd-be12-32bf021b1a31


cls num list(train_dataset):
[400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
cls num list(val_dataset):
[100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]
Epoch: [001] 	 Loss 3.9843 	 Acc 4.24 	 Ac

[I 2022-06-18 01:24:24,896] Trial 0 finished with value: 2.0469346046447754 and parameters: {'n_epoch': 90, 'weight_decay': 0.008360827951486292}. Best is trial 0 with value: 2.0469346046447754.


Acc 2.05 	 AccHead 4.10 	 AccTail 0.02
Epoch: [001] 	 Loss 4.0345 	 Acc 8.72 	 AccHead 6.47 	 AccTail 12.09
Epoch: [002] 	 Loss 3.5585 	 Acc 13.19 	 AccHead 15.51 	 AccTail 9.73
Epoch: [003] 	 Loss 3.3800 	 Acc 16.03 	 AccHead 18.80 	 AccTail 11.89
Epoch: [004] 	 Loss 3.2287 	 Acc 19.02 	 AccHead 20.65 	 AccTail 16.57
Epoch: [005] 	 Loss 3.0985 	 Acc 21.53 	 AccHead 22.71 	 AccTail 19.78
Epoch: [006] 	 Loss 2.9836 	 Acc 22.76 	 AccHead 23.70 	 AccTail 21.37
Epoch: [007] 	 Loss 2.8839 	 Acc 26.91 	 AccHead 29.45 	 AccTail 23.12
Epoch: [008] 	 Loss 2.7733 	 Acc 26.89 	 AccHead 29.99 	 AccTail 22.26
Epoch: [009] 	 Loss 2.6716 	 Acc 30.17 	 AccHead 34.32 	 AccTail 23.99
Epoch: [010] 	 Loss 2.5874 	 Acc 30.47 	 AccHead 32.65 	 AccTail 27.21
Epoch: [011] 	 Loss 2.5114 	 Acc 34.49 	 AccHead 38.71 	 AccTail 28.17
Epoch: [012] 	 Loss 2.4245 	 Acc 36.48 	 AccHead 40.64 	 AccTail 30.29
Epoch: [013] 	 Loss 2.3666 	 Acc 36.79 	 AccHead 41.79 	 AccTail 29.33
Epoch: [014] 	 Loss 2.2695 	 Acc 38.04 	 

...

Acc 8.28 	 AccHead 16.67 	 AccTail 0.02
Epoch: [001] 	 Loss 4.0105 	 Acc 10.01 	 AccHead 12.94 	 AccTail 5.63
Epoch: [002] 	 Loss 3.5326 	 Acc 14.34 	 AccHead 15.21 	 AccTail 13.04
Epoch: [003] 	 Loss 3.3539 	 Acc 16.33 	 AccHead 16.71 	 AccTail 15.77
Epoch: [004] 	 Loss 3.2058 	 Acc 19.93 	 AccHead 22.19 	 AccTail 16.56
Epoch: [005] 	 Loss 3.0587 	 Acc 20.64 	 AccHead 24.95 	 AccTail 14.21
Epoch: [006] 	 Loss 2.9571 	 Acc 23.87 	 AccHead 25.24 	 AccTail 21.82
Epoch: [007] 	 Loss 2.8245 	 Acc 26.55 	 AccHead 29.56 	 AccTail 22.05
Epoch: [008] 	 Loss 2.7308 	 Acc 30.02 	 AccHead 33.72 	 AccTail 24.49
Epoch: [009] 	 Loss 2.6256 	 Acc 30.49 	 AccHead 33.04 	 AccTail 26.70
Epoch: [010] 	 Loss 2.5501 	 Acc 30.93 	 AccHead 35.00 	 AccTail 24.87
Epoch: [011] 	 Loss 2.4782 	 Acc 33.30 	 AccHead 36.11 	 AccTail 29.10
Epoch: [012] 	 Loss 2.4236 	 Acc 35.70 	 AccHead 39.91 	 AccTail 29.41
Epoch: [013] 	 Loss 2.3591 	 Acc 34.98 	 AccHead 39.09 	 AccTail 28.84
Epoch: [014] 	 Loss 2.3087 	 Acc 38.02

[I 2022-06-18 05:20:35,591] Trial 10 finished with value: 10.255349159240723 and parameters: {'n_epoch': 200, 'weight_decay': 0.00048160603724095184}. Best is trial 4 with value: 10.658534049987793.


Acc 10.26 	 AccHead 20.62 	 AccTail 0.04
Epoch: [001] 	 Loss 4.0498 	 Acc 9.16 	 AccHead 11.54 	 AccTail 5.61
Epoch: [002] 	 Loss 3.5286 	 Acc 13.29 	 AccHead 14.32 	 AccTail 11.75
Epoch: [003] 	 Loss 3.3440 	 Acc 16.26 	 AccHead 18.58 	 AccTail 12.79
Epoch: [004] 	 Loss 3.1828 	 Acc 21.00 	 AccHead 23.09 	 AccTail 17.89
Epoch: [005] 	 Loss 3.0565 	 Acc 22.56 	 AccHead 27.04 	 AccTail 15.86
Epoch: [006] 	 Loss 2.9413 	 Acc 25.02 	 AccHead 26.66 	 AccTail 22.56
Epoch: [007] 	 Loss 2.8459 	 Acc 26.22 	 AccHead 30.79 	 AccTail 19.38
Epoch: [008] 	 Loss 2.7572 	 Acc 27.70 	 AccHead 30.64 	 AccTail 23.32
Epoch: [009] 	 Loss 2.6814 	 Acc 28.76 	 AccHead 30.38 	 AccTail 26.35
Epoch: [010] 	 Loss 2.5913 	 Acc 30.23 	 AccHead 32.68 	 AccTail 26.58
Epoch: [011] 	 Loss 2.5242 	 Acc 31.89 	 AccHead 36.18 	 AccTail 25.48
Epoch: [012] 	 Loss 2.4694 	 Acc 35.46 	 AccHead 40.25 	 AccTail 28.31
Epoch: [013] 	 Loss 2.4010 	 Acc 34.68 	 AccHead 37.83 	 AccTail 29.99
Epoch: [014] 	 Loss 2.3692 	 Acc 34.59

[I 2022-06-18 05:55:48,701] Trial 11 finished with value: 10.482787132263184 and parameters: {'n_epoch': 200, 'weight_decay': 0.0006700179119568688}. Best is trial 4 with value: 10.658534049987793.


Acc 10.48 	 AccHead 21.02 	 AccTail 0.10


In [55]:
study = joblib.load('set_100_step_01.pkl')
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.head(10)

,number,value,duration,params_n_epoch,params_weight_decay
0,0,2.046935,0 days 00:15:47.460978,90,0.008361
1,1,8.818360,0 days 00:15:53.215305,90,0.000015
2,2,9.004445,0 days 00:15:52.490003,90,0.000232
3,3,0.920087,0 days 00:15:54.608893,90,0.020430
4,4,10.658534,0 days 00:35:10.138892,200,0.000828
5,5,5.086323,0 days 00:35:09.802480,200,0.008873
6,6,1.974568,0 days 00:15:58.526158,90,0.018706
7,7,8.890726,0 days 00:15:51.404988,90,0.000204
8,8,2.977360,0 days 00:15:57.117649,90,0.006575
9,9,8.280782,0 days 00:35:17.871600,200,0.003237


In [3]:
study = joblib.load('set_10_step_1.pkl')
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.head(12)

,number,value,duration,params_weight_decay
0,0,63.545456,0 days 00:16:38.094424,0.000272
1,1,40.272728,0 days 00:16:49.030045,0.002888
2,2,63.303032,0 days 00:16:46.738419,0.000025
3,3,64.989899,0 days 00:16:35.773399,0.000052
4,4,52.121212,0 days 00:16:32.594171,0.000695
5,5,63.828281,0 days 00:16:32.708094,0.000014
6,6,56.181820,0 days 00:16:37.289671,0.000806
7,7,33.070705,0 days 00:16:40.985060,0.003321
8,8,62.838383,0 days 00:16:23.999368,0.000022
9,9,63.272728,0 days 00:16:45.124516,0.000121


In [4]:
study = joblib.load('set_10_step_01.pkl')
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.head(10)

,number,value,duration,params_n_epoch,params_weight_decay
0,0,35.335155,0 days 00:35:15.375066,200,0.007382
1,1,44.302902,0 days 00:34:58.414487,200,0.000257
2,2,24.699223,0 days 00:35:06.414250,200,0.015852
3,3,42.867252,0 days 00:34:57.689213,200,0.000022
4,4,41.684361,0 days 00:15:56.394683,90,0.000154
5,5,44.646648,0 days 00:35:09.745846,200,0.000288
6,6,41.492268,0 days 00:15:56.506590,90,0.000235
7,7,41.077747,0 days 00:34:48.574028,200,0.002343
8,8,44.211910,0 days 00:35:02.353758,200,0.000421
9,9,38.236782,0 days 00:15:44.087744,90,0.000979


In [63]:
study = joblib.load('set_10_exp_1.pkl')
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.head(12)

,number,value,duration,params_weight_decay
0,0,31.877022,0 days 00:14:06.456049,0.005602
1,1,39.987862,0 days 00:14:11.593239,0.004971
2,2,64.199028,0 days 00:14:16.540441,0.000634
3,3,43.112862,0 days 00:14:14.050488,0.003426
4,4,52.740692,0 days 00:14:17.540152,0.002612
5,5,38.723705,0 days 00:14:16.798418,0.004207
6,6,53.772247,0 days 00:14:07.148492,0.002109
7,7,55.805016,0 days 00:14:13.198820,0.001185
8,8,67.647652,0 days 00:14:08.406153,0.000023
9,9,38.612457,0 days 00:14:02.489840,0.004171


In [62]:
study = joblib.load('set_10_exp_01.pkl')
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.head(12)

,number,value,duration,params_weight_decay
0,0,51.199997,0 days 00:12:15.557371,0.000410
1,1,49.459999,0 days 00:12:02.644975,0.000963
2,2,52.820000,0 days 00:12:10.245198,0.000166
3,3,55.209999,0 days 00:12:04.400424,0.000266
4,4,38.430000,0 days 00:12:12.288648,0.001987
5,5,45.869999,0 days 00:12:21.222020,0.000987
6,6,54.230000,0 days 00:12:17.140612,0.000154
7,7,52.559998,0 days 00:12:26.544858,0.000058
8,8,28.990000,0 days 00:12:19.172162,0.003362
9,9,55.629997,0 days 00:12:17.106273,0.000047


In [61]:
study = joblib.load('set_100_exp_1.pkl')
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.head(12)

,number,value,duration,params_n_epoch,params_weight_decay
0,0,1.033485,0 days 00:31:00.387800,200,0.037809
1,1,8.340223,0 days 00:13:55.552840,90,0.000682
2,2,9.270360,0 days 00:30:53.134421,200,0.000013
3,3,8.691608,0 days 00:14:00.592525,90,0.000073
4,4,9.125672,0 days 00:13:54.098197,90,0.000073
5,5,10.272840,0 days 00:30:27.863838,200,0.000257
6,6,10.572551,0 days 00:30:33.004942,200,0.001405
7,7,8.360893,0 days 00:13:42.935642,90,0.000526
8,8,1.033485,0 days 00:13:18.556360,90,0.068770
9,9,1.033485,0 days 00:13:26.297676,90,0.047246


In [60]:
study = joblib.load('set_100_exp_01.pkl')
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.head(12)

,number,value,duration,params_n_epoch,params_weight_decay
0,0,1.131880,0 days 00:10:21.297956,90,0.029963
1,1,6.085151,0 days 00:23:31.749703,200,0.001043
2,2,5.192108,0 days 00:10:51.839308,90,0.000493
3,3,5.430945,0 days 00:10:51.158423,90,0.000039
4,4,1.038422,0 days 00:23:44.966203,200,0.066359
5,5,6.085151,0 days 00:23:45.929806,200,0.000571
6,6,2.253375,0 days 00:10:17.861273,90,0.022016
7,7,2.585670,0 days 00:10:29.631700,90,0.007837
8,8,4.953271,0 days 00:10:33.087734,90,0.001553
9,9,5.908619,0 days 00:22:57.850075,200,0.000697


In [59]:
study = joblib.load('set_10_step_1.pkl')
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.head(12)

,number,value,duration,params_weight_decay
0,0,63.545456,0 days 00:16:38.094424,0.000272
1,1,40.272728,0 days 00:16:49.030045,0.002888
2,2,63.303032,0 days 00:16:46.738419,0.000025
3,3,64.989899,0 days 00:16:35.773399,0.000052
4,4,52.121212,0 days 00:16:32.594171,0.000695
5,5,63.828281,0 days 00:16:32.708094,0.000014
6,6,56.181820,0 days 00:16:37.289671,0.000806
7,7,33.070705,0 days 00:16:40.985060,0.003321
8,8,62.838383,0 days 00:16:23.999368,0.000022
9,9,63.272728,0 days 00:16:45.124516,0.000121


In [58]:
study = joblib.load('set_10_step_01.pkl')
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.head(12)

,number,value,duration,params_n_epoch,params_weight_decay
0,0,35.335155,0 days 00:35:15.375066,200,0.007382
1,1,44.302902,0 days 00:34:58.414487,200,0.000257
2,2,24.699223,0 days 00:35:06.414250,200,0.015852
3,3,42.867252,0 days 00:34:57.689213,200,0.000022
4,4,41.684361,0 days 00:15:56.394683,90,0.000154
5,5,44.646648,0 days 00:35:09.745846,200,0.000288
6,6,41.492268,0 days 00:15:56.506590,90,0.000235
7,7,41.077747,0 days 00:34:48.574028,200,0.002343
8,8,44.211910,0 days 00:35:02.353758,200,0.000421
9,9,38.236782,0 days 00:15:44.087744,90,0.000979


In [57]:
study = joblib.load('set_100_step_1.pkl')
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.head(12)

,number,value,duration,params_n_epoch,params_weight_decay
0,0,9.794776,0 days 00:36:25.838325,200,0.002314
1,1,5.358624,0 days 00:36:40.001090,200,0.005399
2,2,3.430763,0 days 00:16:31.675795,90,0.003465
3,3,1.036484,0 days 00:16:42.299458,90,0.037093
4,4,1.036484,0 days 00:36:49.319264,200,0.019981
5,5,10.157546,0 days 00:36:59.615525,200,0.001366
6,6,1.886401,0 days 00:16:33.863514,90,0.013638
7,7,1.036484,0 days 00:36:39.593646,200,0.022443
8,8,8.665009,0 days 00:16:31.285765,90,0.000019
9,9,8.665009,0 days 00:16:27.817277,90,0.000015


In [56]:
study = joblib.load('set_100_step_01.pkl')
df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete'], axis=1)
df.head(12)

,number,value,duration,params_n_epoch,params_weight_decay
0,0,2.046935,0 days 00:15:47.460978,90,0.008361
1,1,8.818360,0 days 00:15:53.215305,90,0.000015
2,2,9.004445,0 days 00:15:52.490003,90,0.000232
3,3,0.920087,0 days 00:15:54.608893,90,0.020430
4,4,10.658534,0 days 00:35:10.138892,200,0.000828
5,5,5.086323,0 days 00:35:09.802480,200,0.008873
6,6,1.974568,0 days 00:15:58.526158,90,0.018706
7,7,8.890726,0 days 00:15:51.404988,90,0.000204
8,8,2.977360,0 days 00:15:57.117649,90,0.006575
9,9,8.280782,0 days 00:35:17.871600,200,0.003237
